# 🐮 OSCAR from a Jupyter Notebook: Synchronous Service  

In this example, we will demonstrate how to create and **call a synchronous service**. 

This will showcase how to:  
🔹 Define a synchronous service.  
🔹 Create a synchronous service.   
🔹 Send a request and receive a **real-time response** from the service. 

**_NOTE:_** To do this example, you must have completed the previous step of the tutorial of setting up the environment.

## 🛠️ Service Definition  

To create an [OSCAR service](https://docs.oscar.grycap.net/#concepts) on the cluster, we need to define its configuration using a **YAML** file in **FDL** format.  

We also need:  
📌  🐳 A **Docker image** that includes the application (e.g. a pre-trained AI model) and all the required libraries and data to perform its execution.  
📌 📜 A **shell-script** to be executed inside the container created out of the Docker image for each service invocation.   

📖 Learn more about **FDL** in the [OSCAR documentation](https://docs.oscar.grycap.net/fdl/).  
🎨 For an easier graphical configuration, check out the **[FDL Composer](https://docs.oscar.grycap.net/fdl-composer/)**.    

### 🐮 Cowsay  

For demonstration, we will use a simple **synchronous** service called Cowsay that receives an input in **JSON** and returns a text response 🐄💬:

📥 **INPUT:** 
```json
    {"message": "Hi there"}
```
📤 **OUTPUT:** 
```
     __________
    < Hi there >
     ----------
            \   ^__^
             \  (oo)\_______
                (__)\       )\/\
                    ||----w |
                    ||     ||

```

#### 📜 FDL Definition  

To define a **basic FDL configuration file**, we need to specify:  

📌 **Service Name** → The unique name of the service. 🏷️  
📌 **CPU Cores (`cpu`)** → The number of cores required for execution. ⚡  
📌 **Memory (`memory`)** → The amount of RAM needed (in **Mi** or **Gi**). 💾  
📌 **Docker Image (`image`)** → The container image essential for the service. 🐳  
📌 **Execution Script (`script`)** → The script that will run inside the instance. 📜  

🔹 Additionally, we set the log level to **critical** to avoid including execution logs in the response.

## ⚙️ Service Creation  

After setting up the environment and defining our services, we can now **deploy** them on our **OSCAR cluster**. 🚀  

### 📚 Import Libraries  

To interact with **OSCAR** from a **Jupyter Notebook**, we will use the **[OSCAR Python client library](https://github.com/grycap/oscar_python)**. 

📖 Learn more about it on:  
🔹 [GitHub](https://github.com/grycap/oscar_python) 🛠️  
🔹 [PyPi](https://pypi.org/project/oscar-python/) 📦  

📌 Check the **OSCAR [OpenAPI documentation](https://docs.oscar.grycap.net/api/)** to understand the structure of the returned data. 

**_NOTE:_** This tutorial uses **version 1.3.0** of the Python library. 

In [23]:
from oscar_python.client import Client
import os

### 🔑 Create the Client with Credentials  

To interact with the **OSCAR cluster**, we must create a client instance and provide:  

🔹 **Cluster Identification Name (`cluster_id`)** → Used in the service configuration file to identify the cluster.  
🔹 **OSCAR Cluster Endpoint (`endpoint`)** → The environment variable is set to the current cluster where the notebook is created.  
🔹 **User Identification Tokens (`shortname`)** → Configured during environment setup with the default short name **"oscar-egi"**. 


#### Read the token

In [ ]:
token_path = "/token"
try:
    with open(token_path) as f:
        os.environ["YOUR_TOKEN"] = f.read().strip()
except Exception as err:
    print("Failed with: ", err)

#### Create client

Depending on the type of token you choose to use (**refresh token** or **access token**), you will need to provide different arguments (`refresh_token` or `oidc_token`) when creating the client through which you interact with the OSCAR cluster.


- In this example, we are using an access token, so the token should be provided as the `oidc_token` argument when creating the client.  
- If you use a **refresh token** instead, pass it as the `refresh_token` argument.

In [ ]:
oscar_options = {'cluster_id': 'cluster-id',
                     'endpoint': "http://oscar.oscar.svc.cluster.local:8080",
                     'oidc_token': os.environ["YOUR_TOKEN"], # Change this argument to refresh_token if needed
                     'ssl': 'True'}

client = Client(options=oscar_options)

### 📝 Declare Input Parameters  

We declare the **input parameters** as variables to ensure consistency and avoid errors when using them multiple times.  
Although in this example we will have only one reusable variable. 🔄

In [26]:
serviceName = "cowsay-sync-test"

### 🚀 Create Services  

To deploy services in the cluster, simply call the `create_service()` function and provide **service configuration file**:  

```python
    client.create_service("service_config.yaml")
```

In [27]:
try:
    client.create_service("cowsay-sync.yaml")
    print("Service created")
except Exception as err:
    print("Failed with: ", err)

Service created


#### 📋 Print the List of Available Services  

To check which services are currently deployed in the cluster, use the following command:  

```python
    client.list_services()
```
🔹 This function returns an HTTP response containing a JSON list of available services.

In [28]:
try:
    services = client.list_services()
    print("Services:")
    if services.status_code == 200:
        # Print the name of the services on the cluster
        for value in services.json():
            print("> " + value["name"])
        # You also can print the entire output in JSON format
        # json_object = json.loads(services.text)
        # json_formatted_str = json.dumps(json_object, indent=2)
        # print(json_formatted_str)
except Exception as err:
    print("Failed with: ", err)

Services:
> cowsay-jenkins
> cowsay-secrets
> cowsay-sync-test
> cvat-dextr-demo
> cvat-yolo3-demo
> dog-breed
> imagemagick
> inference
> juno7e3b98
> litter-assessment-test
> myyolov8
> plant-classification-sync
> plants


## 🐄 Execute Cowsay (Synchronous)

To interact with the **Cowsay service**, we need to send a message in **JSON format** using the `run_service()` function,  
providing the **service name** and the **input message** as parameters. 

```python
    client.run_service("service_name", input="input")
```

In [29]:
try:
    # Note that the INPUT for this service must be JSON
    response = client.run_service(serviceName,
                                  input='{"message":"Hi there. I am a cow"}')
    print(response.text)
except Exception as err:
    print("Failed with: ", err)


 ____________________________________
< {"message":"Hi there. I am a cow"} >
 ------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||




---

🔹 We also can specify a **timeout**🕒 to control the execution time and an **output file** in case we want to save the output.💾


In [30]:
try:
    response = client.run_service(serviceName,
                                  input='{"message":"Hi there. You also can find me in output->cowsay-output.txt"}',
                                  output="output/cowsay-output.txt",
                                  timeout=100)
    print(response.text)
except Exception as err:
    print("Failed with: ", err)

 _________________________________________
/ {"message":"Hi there. You also can find \
\ me in output->cowsay-output.txt"}       /
 -----------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||




## 🗑️ Delete Services  

Finally, to delete the services, we simply call the `remove_service()` function, providing the **service name**.  

```python
client.remove_service("service_name")
```

In [31]:
try:
    response = client.remove_service(serviceName)
    print("Service deleted")
except Exception as err:
    print("Failed with: ", err)

Service deleted
